In [61]:
import random
import numpy as np
N=20 #number of items
w=[349,833,684,309,471,680,96,246,872,219,729,253,658,911,941,569,964,480,301,991]  #weights of items
v=[5,187,201,452,691,675,794,559,837,855,17,922,258,21,357,754,957,337,151,616]     #values of items
W=5000 #total capacity  

In [62]:
def generate(v, popSize):
    length = len(v)
    #random.seed(153)
    pop = [[random.randint(0,1) for i in range(length)] for j in range(popSize)]
    return pop


In [63]:
def fitnessFunc(pop, v, w, W):
    fitness = []
    for i in range(len(pop)):
        value = 0
        weight = W+1
        while (weight > W):
            value = 0
            weight = 0
            ones = []
            for j in range(len(pop[i])):
                if pop[i][j] == 1:
                    weight += w[j]
                    value += v[j]
                    ones += [j]
            if weight > W:
                pop[i][ones[random.randint(0, len(ones)-1)]] = 0
        fitness += [value]
    return fitness


In [64]:
'''select best chromosome'''
def selectElite(pop, fit):
    elite = 0
    for i in range(len(fit)):
        if fit[i] > fit[elite]:
            elite = i
    return pop[elite], fit[elite]

In [65]:
'''roulette wheel selection'''
def select(pop, fit):
    size = len(pop)
    totalFit = sum(fit)
    randomNumber = random.randint(0, totalFit)
    tempSum = 0
    parent1 = []
    fit1 = 0
    for i in range(size):
        tempSum += fit[i]
        if tempSum >= randomNumber:
            parent1 = pop.pop(i)   #list.pop(index) -- removes and returns the element at the given index. 
            fit1 = fit.pop(i)      #Returns the rightmost element if index is omitted (roughly the opposite of append()).
            break
    tempSum = 0
    randomNumber = random.randint(0, sum(fit))
    for i in range(len(pop)):
        tempSum += fit[i]
        if tempSum >= randomNumber:
            parent2 = pop[i]
            pop += [parent1]
            fit += [fit1]
    return (parent1, parent2)


In [66]:
def crossover(parent1, parent2,crossover_rate):
    r = np.random.rand()
    if r < crossover_rate:
        randomNumber = random.randint(0, len(parent1)-1)
        child1=parent1[:randomNumber]+parent2[randomNumber:]
        child2=parent2[:randomNumber]+parent1[randomNumber:]
    else:
        child1=parent1
        child2=parent2
    return (child1,child2)

"""parent1=[0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0]
parent2=[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1]
crossover(parent1, parent2,0.85)"""


'parent1=[0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0]\nparent2=[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1]\ncrossover(parent1, parent2,0.85)'

In [67]:
def mutate(gene, mutation_rate):
    for i in range(len(gene)):
        r = np.random.rand()
        if r < mutation_rate:
            gene[i] = bool(gene[i])^1   # 0 to 1 or 1  to 0
    return gene
"""mutate([1, 1, 1, 1, 1, 1, 1, 1, 1], 0.01)"""

'mutate([1, 1, 1, 1, 1, 1, 1, 1, 1], 0.01)'

In [68]:
def newPopulation(pop, fit, crossover_rate, mut):
    popSize = len(pop)
    newPop = []
    newPop += [selectElite(pop, fit)[0]]
    #print "Elite:"
    #print newPop
    while(len(newPop) < popSize):
        (mate1, mate2) = select(pop, fit)
        (child1,child2)= crossover(mate1, mate2, crossover_rate)
        newPop += [mutate(child1, mut)]
        newPop += [mutate(child2, mut)]
    return newPop

In [69]:
!pip install xlsxwriter
def knapsack(v, w, W, N, popSize,crossover_rate, mut, maxGen):
    import xlsxwriter
    workbook = xlsxwriter.Workbook('write_data.xlsx')
    worksheet = workbook.add_worksheet()
    generation = 0
    pop = generate(v, popSize)
    fitness = fitnessFunc(pop, v, w, W)
    while(generation < maxGen):
        generation += 1
        pop = newPopulation(pop, fitness,crossover_rate, mut)
        fitness = fitnessFunc(pop, v, w, W)
        print (fitness)
        print (generation)
        print ("Best individual:" ,selectElite(pop,fitness)[1])
        worksheet.write(generation-1, 0, selectElite(pop,fitness)[1])     
    workbook.close()
    return selectElite(pop, fitness)

In [70]:
knapsack(v, w, W, N, 40, 0.8, 0.01, 100)

[5468, 5331, 3406, 2826, 3396, 3810, 3432, 3290, 3865, 3032, 3032, 3032, 3032, 4636, 3290, 4636, 4636, 3499, 4169, 3954, 3826, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 2989, 3290]
1
Best individual: 5468
[5468, 3290, 3290, 3290, 3290, 3290, 3032, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3627, 3290, 3290, 2333, 2617, 3290, 3290, 3290, 3290, 3290, 3290, 3290]
2
Best individual: 5468
[5468, 3290, 3269, 3290, 3290, 2819, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3290, 3089, 3290, 3290, 3290, 3290, 3290, 3780, 3290, 2461, 3290, 3290, 3290, 3290, 3290, 3290, 2547, 2731]
3
Best individual: 5468
[5468, 3290, 2731, 5468, 3290, 5468, 5468, 4218, 4332, 3729, 4218, 5468, 5468, 2821, 5246, 3290, 3290, 3290, 3290, 3290, 3290, 5183, 2738, 5272, 3084, 5468, 3290, 5468, 5468, 5468, 5468, 

([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0], 7496)